## Melon 주간 Top 100구하기

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
# 하던대로 하면 .. 에러 ... 크롤링해서 매크로(조회수조작)가능할까봐 기계가 접속못하게 막아놓은듯
# 크롤링 차단되었을 때 , User-Agent 구글링 ㅋ

import requests
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
url ='https://www.melon.com/chart/week/index.htm'
html = requests.get(url, headers = headers).text

In [3]:
soup =  BeautifulSoup(html, 'html.parser')

## 멜론주간 F12에서 주간 시작일~ 끝나는일

In [4]:
soup.select_one('.yyyymmdd').get_text()
#양끝 지저분한건 strip하자

'\r\n\t\t\t\t\t\t\t2020.11.09 ~ 2020.11.15\r\n\t\t\t\t\t\t'

In [5]:
date = soup.select_one('.yyyymmdd').get_text().strip() #양끝 지저분한건 strip하자
date  #깔끔^-^

'2020.11.09 ~ 2020.11.15'

In [6]:
start_day = date[:10]
end_day = date[13:]
start_day, end_day

('2020.11.09', '2020.11.15')

## 곡 리스트 찾기

In [7]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

In [8]:
#찾기1_순위rank (전주대비)등락diff 제목title 가수singer 앨범명album 좋아요like-count
# 중 순위rank
rank = int(trs[0].select_one('.rank').string)
rank

1

In [9]:
span = trs[1].select_one('.rank_wrap')
span

<span class="rank_wrap" title="1단계 상승">
<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>
<span class="up">1</span>
</span>

In [10]:
#찾기2_(전주대비)등락diff
for i in [0, 1, 2, 19]:  #샘플 유지,업,다운,신규진입20번째(19)
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100 #print('신규진입')은 -100으로 두겠다
    if len(spans) == 3:  #3가지경우
        diff = int(spans[2].string)

        if diff != 0:   #0이 아니면 사인을 확인함(올랐는지안올랐는지)
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1  #하락했을때 -1로 변경
    print(diff)

0
1
-1
-100


In [11]:
span = trs[0].select_one('.rank_wrap')
span.find_all('span')         #3가지

[<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>,
 <span class="none">순위 동일</span>,
 <span class="none">0</span>]

In [12]:
span = trs[1].select_one('.rank_wrap')
span.find_all('span')        #3가지

[<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>,
 <span class="none">단계 상승</span>,
 <span class="up">1</span>]

In [13]:
span = trs[19].select_one('.rank_wrap')
span.find_all('span')        #2가지

[<span class="bullet_icons rank_new"><span class="none">순위 진입</span></span>,
 <span class="none">순위 진입</span>]

In [14]:
#찾기3,4_제목title 가수singer
info = trs[0].select_one('.wrap_song_info')
a_list = info.find_all('a')
a_list          #타이틀 가수 순으로 나옴

[<a href="javascript:melon.play.playSong('19041301',32872978);" title="Dynamite 재생">Dynamite</a>,
 <a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a>,
 <a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a>]

In [15]:
title = a_list[0].string
singer = a_list[1].string
title, singer

('Dynamite', '방탄소년단')

## 다른방법 리팩토링+_+
위방법으로 찾지말고 ellipsis rank01 02 03으로 다시 찾으신다함

In [16]:
#같은결과_다른방법1
info = trs[0].select_one('.ellipsis.rank01') #클래스명
title = info.find('a').string
info = trs[0].select_one('.ellipsis.rank02')
singer = info.find('a').string
info = trs[0].select_one('.ellipsis.rank03')
album = info.find('a').string

title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

In [17]:
#같은결과_다른방법2   #제목title 가수singer 앨범명album 찾았다
info = trs[0].select('.ellipsis') #3개나옴
title = info[0].find('a').string
singer = info[1].find('a').string
album = info[2].find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

In [18]:
#찾기6_좋아요like-count  class=cnt에 있다
cnt = trs[0].select_one('.cnt')
cnt.get_text()    #0가 나왔는데.. 웹에는 숫자가있단말이다!!!
#뷰티풀숩으로는 0을 구해낼수없는데
#셀레늄으로는 가능함.... 그래서 좋아요 뺍니다~_~;;;;

'\n총건수\r\n\t\t\t\t\t\t\t\t\t\t\t\t0\r\n\t\t\t\t\t\t\t\t\t\t\t'

## Melon 주간 Top 100 구한 결과로 
## 데이프레임 만들기

In [19]:
# trs rank diff title singer album
ranks = []; diffs = []; titles = []; singers = []; albums = []
service_list = soup.select_one('.service_list_song') #리스트 찾기부분
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [20]:
for tr in trs :
    # print(i) 에러났을때. 어디서 에러났나 찾을때 필요
    rank = int(tr.select_one('.rank').string)              #복붙1

    info = tr.select_one('.ellipsis.rank01')        #클래스명 #복붙
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string

    entry = tr.select_one('.rank_wrap')              #복붙3
    spans = entry.find_all('span')
    diff = -100                                 #print('신규진입')은 -100으로 두겠다
    if len(spans) == 3:                         #3가지경우
        diff = int(spans[2].string)
        if diff != 0:                       #0이 아니면 사인을 확인함(올랐는지안올랐는지)
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1                   #하락했을때 -1로 변경
    ranks.append(rank)
    diffs.append(diff)
    titles.append(title)
    singers.append(singer)
    albums.append(album)

In [21]:
#데이터프레임만들기
top100 = pd.DataFrame({
    'rank': ranks,
    'diff': diffs,
    'title': titles,
    'singer': singers,
    'album': albums 
})
top100 = top100.set_index('rank')
print(f' 기간: {start_day} ~ {end_day}')
top100.head()

 기간: 2020.11.09 ~ 2020.11.15


,diff,title,singer,album
rank,,,,
1,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
3,-1,Lovesick Girls,BLACKPINK,THE ALBUM
4,1,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)
5,-1,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME


In [22]:
top100.to_csv('top100.csv', sep=',', encoding='utf8')  